In [1]:
import pandas as pd
import sqlite3

In [2]:
# conectamos con la base de datos my_database.db
connection = sqlite3.connect("my_database.db")

# obtenemos un cursor que utilizamos para las queries
crsr = connection.cursor()

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [4]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

In [6]:
query = '''
CREATE TABLE IF NOT EXISTS reparto (
	CodE VARCHAR(2) NOT NULL,
	CodB VARCHAR(3) NOT NULL,
	CodC VARCHAR(2) NOT NULL,
	Fecha DATE NOT NULL,
	Cantidad SMALLINT,
	PRIMARY KEY (CodE,CodB,CodC,fecha)
);
'''
crsr.execute(query)

In [8]:
query = '''
CREATE TABLE IF NOT EXISTS cervezas (
    CodC VARCHAR(2),
    Envase VARCHAR(32),
    Capacidad FLOAT(2),
    Stock INT(5),
    PRIMARY KEY (CodC)
)
'''

crsr.execute(query)

In [9]:
query = '''
CREATE TABLE IF NOT EXISTS bares (
    CodB VARCHAR(2),
    Nombre VARCHAR(32),
    Cif VARCHAR(32),
    Localidad VARCHAR(32),
    PRIMARY KEY (CodB)
)
'''

crsr.execute(query)

In [10]:
query = '''
CREATE TABLE IF NOT EXISTS empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(32),
    Sueldo INT(5),
    PRIMARY KEY (CodE)
)
'''

crsr.execute(query)

In [7]:
query = '''
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) 
VALUES  ('1','001','01','2005-10-21',240 ),
( '1','001','02','2005-10-21',48 ),
( '1','002','03','2005-10-22',60 ),
( '1','004','05','2005-10-22',4 ),
( '2','002','03','2005-10-22',48 ),
( '2','002','05','2005-10-23',2 ),
( '2','004','01','2005-10-23',480 ),
( '2','004','02','2005-10-24',72 ),
( '3','003','03','2005-10-24',48 ),
( '3','003','04','2005-10-25',20 )
;
'''
crsr.execute(query)

In [11]:
query = '''
INSERT OR REPLACE INTO bares VALUES ('001','Stop', '11111111X','Villa Botijo'),
('002', 'Las Vegas', '22222222Y', 'Villa Botijo'),
('003', 'Club Social', NULL, 'Las Ranas'),
('004','Otra Ronda','33333333Z','La Esponja')
'''
crsr.execute(query)

In [12]:
query = '''
INSERT OR IGNORE INTO Cervezas VALUES ('01','Botella',0.2,3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04','Botella',1,288),
('05','Barril',60,30)
'''
crsr.execute(query)

In [13]:
query = '''
INSERT OR REPLACE INTO empleados VALUES ('1','Prudencio Caminero',120000),
('2', 'Vicente Merario', 110000),
('3', 'Valentin Siempre', 100000)
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [14]:
query = '''
SELECT * FROM reparto 
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20
